[View in Colaboratory](https://colab.research.google.com/github/wronnyhuang/word_embed_demo/blob/master/Large_Operations_on_word_vectors_v2.ipynb)

# Operations on word vectors

Welcome to your first assignment of this week! 

Because word embeddings are very computionally expensive to train, most ML practitioners will load a pre-trained set of embeddings. 

**After this assignment you will be able to:**

- Load pre-trained word vectors, and measure similarity using cosine similarity
- Use word embeddings to solve word analogy problems such as Man is to Woman as King is to ______. 
- Modify word embeddings to reduce their gender bias 

Let's get started! Run the following cell to load the packages you will need.

In [1]:
# clone the git repo with all the helper code and images
! rm -rf * .*
! git init
! git remote add origin https://github.com/wronnyhuang/word_embed_demo
! git fetch
! git checkout -t origin/master

rm: refusing to remove '.' or '..' directory: skipping '.'
rm: refusing to remove '.' or '..' directory: skipping '..'
Initialized empty Git repository in /content/.git/
remote: Counting objects: 69, done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 69 (delta 32), reused 28 (delta 6), pack-reused 0
Unpacking objects: 100% (69/69), done.
From https://github.com/wronnyhuang/word_embed_demo
 * [new branch]      master     -> origin/master
Branch master set up to track remote branch master from origin.
Already on 'master'


Next, lets load the word vectors. For this assignment, we will use 50-dimensional GloVe vectors to represent words. Run the following cell to load the `word_to_vec_map`. 

In [3]:
# download large pretrained glove embeddings (5-min donwload time)
! wget http://nlp.stanford.edu/data/glove.840B.300d.zip
! unzip glove.840B.300d.zip # unzip large pretrained glove embedding zip file (~3 min)

Archive:  glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


In [4]:
import numpy as np
from w2v_utils import *

Using TensorFlow backend.


In [0]:
# read glove embedding
words, word_to_vec_map = read_glove_vecs('glove.840B.300d.txt')

Ignored: . . . -0.1573 -0.29517 0.30453 -0.54773 0.098293 -0.1776 0.21662 0.19261 -0.21101 0.53788 -0.047755 0.40675 0.023592 -0.32814 0.046858 0.19367 0.25565 -0.021019 -0.15957 -0.1023 0.20303 -0.043333 0.11618 -0.18486 0.0011948 -0.052301 0.34587 0.052335 0.16774 -0.21384 0.055947 0.24934 -0.12179 0.16749 0.28922 -0.033739 0.3015 -0.13241 0.092635 0.37155 -0.2884 -0.0052731 -0.001005 -0.51153 -0.28476 -0.20139 0.11837 -0.0055891 0.43604 0.16796 -0.2701 0.063957 -0.093253 -0.22079 0.36501 0.06545 0.23941 -0.19292 0.098293 0.12172 -0.1168 -0.027436 0.20507 -0.39139 -0.23111 0.46239 0.22888 -0.028415 -0.1798 0.23817 0.28093 -0.47935 0.23177 -0.35587 0.14246 0.11861 0.011018 0.091986 0.0054809 -0.39955 -0.40183 -0.10629 -0.30851 0.12383 -0.16737 -0.43569 0.4211 -0.57416 -0.19964 0.51312 0.090747 -0.21657 0.043519 0.24288 0.081134 0.49104 -0.33342 -0.31056 -0.3136 0.26931 -0.14402 0.33185 -0.21662 -0.072985 0.080603 -0.7266 -0.098385 -0.36233 -0.25346 0.1154 0.25738 0.15802 -0.15633 -0.0

Ignored: to name@domain.com 0.33865 0.12698 -0.16885 0.55476 0.48296 0.45018 0.0094233 -0.36575 -0.87561 -0.35802 0.2379 0.31284 -0.081367 0.061482 0.81921 0.77488 0.68518 -0.48005 -0.012098 0.53366 0.038321 0.26857 0.56736 0.20427 0.2847 0.68113 -0.26921 0.10099 -0.33252 -0.22999 0.66003 0.21833 -0.086523 -0.30044 -0.42253 0.47525 -0.2165 0.3268 0.63515 -0.15657 -0.25835 -0.11663 -0.41092 -0.73779 -0.0015122 0.14481 0.13287 0.26097 0.58175 0.29285 0.27168 -0.0058512 0.27731 -0.40565 0.05047 0.059203 -0.39081 -0.098029 -0.13969 0.42714 -0.20103 -0.019703 1.1957 0.36278 -0.6468 0.096856 -0.54383 0.29666 -0.0098302 0.5042 -0.3419 -0.25067 0.72375 0.81957 0.57959 -0.28619 0.91511 -0.49127 0.42129 0.11429 -0.32411 -0.092257 -1.0342 -0.25774 -0.19044 -0.34201 1.2339 -0.65392 0.83096 -0.3133 -0.10256 -0.075641 0.88435 -0.1656 0.041536 0.23504 -0.59116 -0.12573 0.48002 0.23561 -0.16167 -0.1404 -0.45722 -0.13186 0.9173 -0.78831 -0.027372 0.036396 -0.49337 -0.2556 -0.15139 -0.53403 0.90196 0.60

Ignored: U.S.  0.067189 0.076706 0.77225 -0.50427 0.4329 -0.3829 -0.3535 0.3707 -0.099857 0.21221 -0.071712 0.10006 0.020111 0.1339 0.2438 0.37665 0.52001 -0.94182 0.51813 0.15695 0.12436 0.48902 0.31168 0.14448 -0.88846 -0.31694 0.034804 -0.10545 -0.71586 0.014892 0.52007 -0.39677 0.3021 -0.14678 0.36627 0.066181 0.35821 -0.014203 0.22382 0.71554 0.2794 0.32126 0.52007 -0.28943 0.015897 -0.098903 -0.0046503 -0.49874 0.25548 0.11274 -0.028864 -0.15556 0.084751 -0.42332 0.082075 -0.48425 0.25567 -0.049282 0.25536 -0.313 -0.010674 -0.35317 0.21204 -0.21661 -0.28918 0.14151 0.21796 0.40517 0.20879 0.14477 0.13771 0.10752 0.34308 0.29546 -0.19086 0.21193 -0.39707 0.068634 -0.022573 -0.068831 -0.1052 -0.069845 -0.22591 0.31867 -0.025092 -0.028913 -0.079526 -0.32293 -0.086526 -0.20378 -0.39259 -0.26203 -0.17283 0.32912 0.048619 0.66813 0.27227 0.25184 0.52969 0.51961 -0.67491 0.1628 0.35931 0.24268 0.14301 -1.6812 0.12558 -0.65106 -0.73602 0.10712 0.14134 0.1059 -0.013123 0.12343 0.26233 0.7

Ignored: D.  -0.12984 -0.13879 -0.068701 -0.37455 -0.42556 -0.1573 -0.21617 0.66126 0.14478 -0.67518 0.49215 -0.13621 -0.0079129 0.81389 0.10494 0.010292 0.68389 -0.55652 -0.060479 -0.32342 -0.16138 0.059715 -0.079568 0.40071 -0.53467 -0.14671 -0.038443 -0.14106 -0.016323 -0.44 0.10207 -0.28313 0.1799 0.066779 0.10502 0.18622 0.012381 0.10765 0.17676 -0.10866 0.2624 0.21555 -0.094971 -0.63723 -0.35377 -0.47818 -0.27045 -0.34995 0.017105 0.19444 0.28321 0.16196 0.61037 0.077094 0.34793 -0.3836 -0.10492 -0.095039 0.21358 -0.55143 -0.22054 -0.13888 0.52093 0.12044 -0.73315 0.57874 -0.14287 1.0882 0.14659 0.072178 0.0044069 -0.18223 0.90194 -0.03214 -0.15608 -0.13517 -0.39633 0.25829 0.49077 -0.20453 0.001487 -0.23409 -0.34901 0.027488 -0.070828 0.13056 0.43122 -0.70908 -0.42944 -0.069644 0.078926 -0.40884 -0.56928 -0.12663 0.07812 0.94638 0.13775 -0.0256 0.22136 0.64709 0.25631 0.1931 1.1177 0.15844 0.079361 -0.55323 0.56123 -0.42342 -0.52872 -0.69732 0.2338 -0.17412 -0.2344 -0.24215 0.29

Ignored:    -0.38024 0.61431 0.81146 -0.76394 -0.19657 0.11078 -0.48388 0.20633 0.29338 -1.1915 0.031847 -0.4115 0.36805 0.49588 0.13105 -0.35901 0.66379 -1.1519 0.067329 0.044183 0.47607 -0.01831 -0.094304 0.72384 -1.0576 -0.32641 -0.60887 -0.23653 0.53253 0.0071736 0.14743 -0.39498 0.22907 0.40914 -0.29576 0.37041 0.41462 0.32433 0.013848 0.1493 0.10169 0.26288 -0.089235 -0.10305 -0.076485 0.14427 0.13496 -0.20195 -0.45063 0.58541 0.43103 -0.57936 0.22184 -0.19762 -0.032769 -1.1831 -0.30654 -0.46583 -0.0965 -0.37734 -0.27668 0.13681 1.1758 0.090845 -1.2651 0.27605 0.3203 0.27979 0.002744 -0.12 0.17253 -0.44621 -0.044658 -0.35172 0.12478 -0.24757 -0.51956 0.49587 -0.21825 -0.33698 -0.40646 -0.45802 0.033569 0.29989 0.17406 0.12293 0.99174 -0.84194 0.26894 -0.10951 -0.14404 -0.50987 0.0015717 -0.29964 0.18028 0.58709 0.617 -0.10019 -0.03308 -0.080183 0.49027 0.13914 0.78463 0.40423 0.0052089 -0.42332 0.53297 -0.62549 -0.49874 -0.40964 0.17438 -0.11739 0.45287 0.073891 0.76254 0.47945 -

...

Ignored: c.       0.69702 -0.50371 0.47688 -0.96898 -0.76444 0.34435 -0.079375 0.43974 0.32278 -1.5106 0.75813 0.095151 -0.08132 0.57253 0.2043 -0.13842 0.28406 -1.5951 0.25111 0.26001 0.21556 -0.073268 0.68646 -0.019362 -0.58656 0.22098 -0.25113 0.038717 -0.35792 -0.1937 -0.27152 -0.63314 0.51057 -0.06598 0.072477 0.2161 0.56454 0.32641 0.31093 0.29504 0.41816 0.13331 0.29507 -0.26595 -0.18905 -0.012236 -0.23497 -0.10556 -0.1951 0.48872 0.37805 -0.044137 0.60225 -0.3065 -0.087954 -0.40117 0.116 -0.48059 0.1659 0.14231 -0.65066 -0.14146 0.95906 -0.41714 -1.1811 0.34955 -0.030244 -0.016591 -0.14267 -0.43776 -0.028881 0.14109 0.22326 -0.212 -0.077685 -0.20007 -0.5327 0.47307 -0.26991 -0.60577 -0.53204 -0.23876 -0.22139 -0.51206 -0.44476 0.30419 0.44518 -0.23326 -0.52177 -0.42644 0.25173 -0.045191 0.34412 0.030814 0.17025 1.2334 0.52235 -0.018989 0.4211 0.34274 -0.097579 0.10797 0.40787 -0.039164 0.035022 -1.1511 0.2363 -0.2879 -0.3383 -0.2785 -0.019554 0.21262 0.0061783 -0.030352 0.05150

Ignored: C.   0.48609 -0.16911 1.1393 -0.74215 -0.70672 0.010331 0.017448 0.83719 0.2868 -1.3985 0.82929 -0.097194 -0.16109 0.62767 -0.46766 -0.26396 0.35134 -1.0469 0.52911 0.2194 -0.047382 -0.011144 -0.03453 0.46724 -0.13235 0.21734 -0.48548 -0.17504 -0.47163 -0.58426 0.184 0.30181 0.074207 0.40001 -0.59305 -0.34403 0.4831 0.63993 0.6981 0.2394 0.70319 0.12749 0.21413 -0.35618 0.25246 0.13595 -0.0086659 0.12942 -0.34676 0.25261 0.56622 0.23803 0.4053 0.17352 0.23978 -0.29886 -0.0090605 -0.56857 0.091584 -0.031941 0.14896 0.20458 0.93652 -0.047067 -0.8523 0.38912 -0.31209 0.40012 -0.097924 0.078413 -0.74872 -0.12935 0.59062 0.10334 -0.25928 0.33781 -0.12193 0.32685 0.2842 -0.30802 -0.59945 -0.63248 -0.30119 0.22084 -0.78138 -0.10736 0.36207 -0.51556 -0.070495 -0.18542 0.45218 -0.42122 -0.20386 -0.14649 0.17272 1.3441 0.12317 -0.26233 0.12126 0.2792 -0.057691 -0.046662 0.75898 0.021826 0.37546 -0.54858 0.32141 -0.64615 0.016288 -0.99849 -0.03137 0.089353 -0.30316 -0.24963 0.12662 0.108

Ignored: U.S.A.  0.56949 -0.059926 0.39739 -0.42354 -0.053781 -0.25468 -0.10253 0.44728 -0.047639 -1.0085 -0.20099 0.0040873 0.10335 0.32674 -0.0074714 -0.050848 0.75791 -1.5813 0.92739 0.29191 0.251 -0.054366 0.41058 -0.030448 -0.70679 -0.11268 0.022423 -0.0023303 -0.89066 -0.38642 0.48431 -0.22315 0.20092 -0.23476 0.24204 -0.15965 0.037377 0.47492 0.20357 0.90049 -0.4683 0.64698 0.0062603 -0.0087442 -0.057709 -0.15195 0.61658 -0.90464 0.39071 0.31374 0.47867 -0.2507 0.20725 -0.19726 0.047897 -0.49449 0.16456 0.14669 0.36545 -0.39859 -0.33831 -0.09366 0.65108 -0.15194 -0.58484 0.43457 -0.01696 0.38338 -0.28108 -0.22743 -0.43451 0.20367 -0.0051644 0.47891 -0.39917 0.26673 -0.49094 0.5217 0.027377 0.063008 -0.19422 -0.00916 -0.12271 0.44399 -0.47383 -0.33731 0.16024 -0.3439 -0.25565 -0.33256 -0.01071 -0.16811 -0.085847 0.64896 -0.47099 1.2247 0.58541 0.44337 0.021696 0.30342 0.071962 -0.071072 0.32175 0.16016 0.10647 -1.007 0.3015 -0.36073 -0.3021 -0.12103 0.035356 0.047967 -0.016034 -0

Ignored: at  name@domain.com 0.44321 -0.40005 -0.20065 1.1209 0.34041 0.086082 -0.067128 0.0022702 -0.94649 -1.4669 0.61248 0.34827 -0.20983 -0.61434 0.41102 0.57759 0.69071 -1.9301 0.75265 -0.13238 0.22003 0.28856 0.35234 0.45989 -0.21944 0.1931 -0.11664 0.14996 0.70354 -0.039238 0.55298 -0.53503 -0.3221 -0.28595 -0.1246 0.054544 -0.45937 0.1447 0.8203 -0.33182 0.10864 -0.56552 0.39898 -0.65012 -0.20285 0.11557 0.35711 -0.23958 -0.30281 0.51593 0.71883 -0.30403 0.59458 -0.3217 -0.23967 -0.2576 -0.50224 -0.36055 -0.71763 0.4981 -0.69945 -0.0072578 0.37327 -0.029839 -0.42705 0.93128 -0.046928 0.045162 -0.44879 0.16579 -0.26272 -0.35286 0.17395 -0.24436 -0.1439 -0.39857 0.25342 -0.44737 0.37618 -0.80252 -0.87776 -0.19282 -0.48746 0.065159 -0.24349 -0.77669 0.81629 -0.043888 -0.68276 -0.15709 -0.46533 -0.066009 0.063028 0.090332 -0.81297 0.88979 -0.6391 0.17351 0.3328 -0.30808 0.46158 -0.11289 -0.0261 -0.089243 0.37318 -0.73511 0.19798 -0.060219 -0.12113 -0.2146 0.62061 0.34296 0.89595 0.

Ignored: Figs.  -0.31426 -0.23576 0.54268 -0.21716 0.3814 -0.3051 0.17385 0.3059 1.019 -1.8186 1.11 0.49289 0.0021534 -0.32818 -0.15761 -0.13406 0.6194 -1.0473 0.19271 -0.42464 0.036757 -0.17887 0.1797 0.49294 -0.63403 0.17953 -0.69384 0.2165 -0.49612 0.6059 -0.19376 -0.19268 -0.12885 0.61254 -0.015287 -0.4951 0.25521 0.28779 -0.37875 -0.15028 -0.52467 -0.18398 -0.33977 0.02258 -0.32644 -0.6402 0.16081 -0.6652 0.12273 -0.11284 0.39886 -0.25239 0.035853 -0.65371 -0.14495 -0.90472 0.53364 0.1606 -0.73009 0.098122 0.14343 -0.63542 -0.55077 -0.98423 -1.1359 -0.16536 0.33362 -0.20724 0.4876 -0.37738 -0.070767 0.60466 0.4568 -0.76762 -0.14465 -0.22076 0.13883 0.037298 0.90607 0.082264 -0.084899 0.36646 0.14177 0.56119 -0.03813 0.31532 0.78671 -0.59949 -0.58141 0.15117 -0.38007 0.44113 -0.21346 -0.25893 0.54702 0.65797 0.34739 0.61559 0.059379 -0.36657 0.3008 -0.23099 0.092033 0.10072 -0.50131 -0.86915 -0.293 -0.39202 -0.38608 -0.19558 -0.45596 0.041117 -0.051141 0.46856 0.028241 0.40429 0.64

You've loaded:
- `words`: set of words in the vocabulary.
- `word_to_vec_map`: dictionary mapping words to their GloVe vector representation.

You've seen that one-hot vectors do not do a good job cpaturing what words are similar. GloVe vectors provide much more useful information about the meaning of individual words. Lets now see how you can use GloVe vectors to decide how similar two words are. 



# 1 - Cosine similarity

To measure how similar two words are, we need a way to measure the degree of similarity between two embedding vectors for the two words. Given two vectors $u$ and $v$, cosine similarity is defined as follows: 

$$\text{CosineSimilarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

where $u.v$ is the dot product (or inner product) of two vectors, $||u||_2$ is the norm (or length) of the vector $u$, and $\theta$ is the angle between $u$ and $v$. This similarity depends on the angle between $u$ and $v$. If $u$ and $v$ are very similar, their cosine similarity will be close to 1; if they are dissimilar, the cosine similarity will take a smaller value. 

<img src="https://github.com/wronnyhuang/word_embed_demo/blob/master/images/cosine_sim.png?raw=1" style="width:800px;height:250px;">
<caption><center> **Figure 1**: The cosine of the angle between two vectors is a measure of how similar they are</center></caption>

**Exercise**: Implement the function `cosine_similarity()` to evaluate similarity between word vectors.

**Reminder**: The norm of $u$ is defined as $ ||u||_2 = \sqrt{\sum_{i=1}^{n} u_i^2}$

In [0]:
# GRADED FUNCTION: cosine_similarity

def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    ### START CODE HERE ###
    # Compute the dot product between u and v (≈1 line)
    dot = np.dot(u,v)
    # Compute the L2 norm of u (≈1 line)
    norm_u = np.linalg.norm(u)
    # Compute the L2 norm of v (≈1 line)
    norm_v = np.linalg.norm(v)
    # Compute the cosine similarity defined by formula (1) (≈1 line)
    cosine_similarity = dot/(norm_u*norm_v)
    ### END CODE HERE ###
    
    return cosine_similarity

In [8]:
father = word_to_vec_map["father"]
mother = word_to_vec_map["mother"]
ball = word_to_vec_map["ball"]
crocodile = word_to_vec_map["crocodile"]
france = word_to_vec_map["france"]
italy = word_to_vec_map["italy"]
paris = word_to_vec_map["paris"]
rome = word_to_vec_map["rome"]

print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))
print("cosine_similarity(ball, crocodile) = ",cosine_similarity(ball, crocodile))
print("cosine_similarity(france - paris, rome - italy) = ",cosine_similarity(france - paris, rome - italy))

cosine_similarity(father, mother) =  0.8298245273848204
cosine_similarity(ball, crocodile) =  0.11947030012391592
cosine_similarity(france - paris, rome - italy) =  -0.34433750699397075


**Expected Output**:

<table>
    <tr>
        <td>
            **cosine_similarity(father, mother)** =
        </td>
        <td>
         0.890903844289
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(ball, crocodile)** =
        </td>
        <td>
         0.274392462614
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(france - paris, rome - italy)** =
        </td>
        <td>
         -0.675147930817
        </td>
    </tr>
</table>

After you get the correct expected output, please feel free to modify the inputs and measure the cosine similarity between other pairs of words! Playing around the cosine similarity of other inputs will give you a better sense of how word vectors behave. 

## 2 - Word analogy task

In the word analogy task, we complete the sentence <font color='brown'>"*a* is to *b* as *c* is to **____**"</font>. An example is <font color='brown'> '*man* is to *woman* as *king* is to *queen*' </font>. In detail, we are trying to find a word *d*, such that the associated word vectors $e_a, e_b, e_c, e_d$ are related in the following manner: $e_b - e_a \approx e_d - e_c$. We will measure the similarity between $e_b - e_a$ and $e_d - e_c$ using cosine similarity. 

**Exercise**: Complete the code below to be able to perform word analogies!

In [0]:
# GRADED FUNCTION: complete_analogy

def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    Performs the word analogy task as explained above: a is to b as c is to ____. 
    
    Arguments:
    word_a -- a word, string
    word_b -- a word, string
    word_c -- a word, string
    word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    
    # convert words to lower case
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    ### START CODE HERE ###
    # Get the word embeddings v_a, v_b and v_c (≈1-3 lines)
    e_a, e_b, e_c = word_to_vec_map[word_a],word_to_vec_map[word_b],word_to_vec_map[word_c]
    
    ### END CODE HERE ###
    
    words = word_to_vec_map.keys()
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_word = None                   # Initialize best_word with None, it will help keep track of the word to output

    # loop over the whole word vector set
    for w in words:        
        # to avoid best_word being one of the input words, pass on them.
        if w in [word_a, word_b, word_c]:
            continue
        
        ### START CODE HERE ###
        # Compute cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)  (≈1 line)
#         print(len(e_a), len(e_b), len(e_c))
        if len(e_a)!=300 or len(e_b)!=300 or len(e_c)!=300 or len(word_to_vec_map[w])!=300: continue
        cosine_sim = cosine_similarity(e_b-e_a, word_to_vec_map[w]-e_c)
        
        # If the cosine_sim is more than the max_cosine_sim seen so far,
            # then: set the new max_cosine_sim to the current cosine_sim and the best_word to the current word (≈3 lines)
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w
        ### END CODE HERE ###
        
    return best_word

Run the cell below to test your code, this may take 1-2 minutes.

In [21]:
triads_to_try = [('italy', 'italian', 'spain'),
                 ('india', 'delhi', 'japan'),
                 ('man', 'woman', 'boy'),
                 ('small', 'smaller', 'large'),
                 ('man','woman','king'),
                 ('hot','cold','pleasure'),
                 ('man','god','woman'),
                 ('scissors','paper','knife'),
                 ('eyes','glasses','head'),
                 ('swimmer','swim','runner'),
                 ('cat','cats','deer'),
                 ('actor','actress','hero'),
                 ('car','drive','boat'),
                 ('life','death','joy'),
                 ('dog','puppy','cat'),
                 ('speak','sing','walk'),
                 ('chef','food','sculptor'),
                 ('bees','hive','bears'),
                 ('like','love','dislike'),
                 ('mansion','shack','yacht'),
                 ('hammer','nail','comb'),
                 ('sunglasses', 'sun', 'goggles'),
                 ('snake','slither','horse'),
                 ('cat','meow','dog'),
                 ('moon','night','sun'),
                 ('gray','black','pink'),
                 ('pencil','write','scissors'),
                 ('ski','snow','raft'),
                 ('one','penny','five'),
                 ('sing','sang','drink'),
                 ('sound','ear','aroma'),
                 ('referee','soccer','umpire'),
                 ('apple','fruit','beef'),
                 ('actor','act','artist')]
for triad in triads_to_try:
    print ('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad, word_to_vec_map)))

italy -> italian :: spain -> spanish
india -> delhi :: japan -> vihar
man -> woman :: boy -> girl
small -> smaller :: large -> larger
man -> woman :: king -> queen
hot -> cold :: pleasure -> coldness
man -> god :: woman -> goddess
scissors -> paper :: knife -> papers
eyes -> glasses :: head -> Glasses
swimmer -> swim :: runner -> walk
cat -> cats :: deer -> coyotes
actor -> actress :: hero -> heroine
car -> drive :: boat -> drives
life -> death :: joy -> deaths
dog -> puppy :: cat -> kitten
speak -> sing :: walk -> singing
chef -> food :: sculptor -> foods
bees -> hive :: bears -> workspace
like -> love :: dislike -> adore
mansion -> shack :: yacht -> Shack
hammer -> nail :: comb -> Nail
sunglasses -> sun :: goggles -> sunlight
snake -> slither :: horse -> sidle
cat -> meow :: dog -> woof
moon -> night :: sun -> afternoon
gray -> black :: pink -> lesbian
pencil -> write :: scissors -> read
ski -> snow :: raft -> rain
one -> penny :: five -> pennies
sing -> sang :: drink -> drank
sound 

**Expected Output**:

<table>
    <tr>
        <td>
            **italy -> italian** ::
        </td>
        <td>
         spain -> spanish
        </td>
    </tr>
        <tr>
        <td>
            **india -> delhi** ::
        </td>
        <td>
         japan -> tokyo
        </td>
    </tr>
        <tr>
        <td>
            **man -> woman ** ::
        </td>
        <td>
         boy -> girl
        </td>
    </tr>
        <tr>
        <td>
            **small -> smaller ** ::
        </td>
        <td>
         large -> larger
        </td>
    </tr>
</table>

Once you get the correct expected output, please feel free to modify the input cells above to test your own analogies. Try to find some other analogy pairs that do work, but also find some where the algorithm doesn't give the right answer: For example, you can try small->smaller as big->?.  

### Congratulations!

You've come to the end of this assignment. Here are the main points you should remember:

- Cosine similarity a good way to compare similarity between pairs of word vectors. (Though L2 distance works too.) 
- For NLP applications, using a pre-trained set of word vectors from the internet is often a good way to get started. 

Even though you have finished the graded portions, we recommend you take a look too at the rest of this notebook. 

Congratulations on finishing the graded portions of this notebook! 


## 3 - Debiasing word vectors (OPTIONAL/UNGRADED) 

In the following exercise, you will examine gender biases that can be reflected in a word embedding, and explore algorithms for reducing the bias. In addition to learning about the topic of debiasing, this exercise will also help hone your intuition about what word vectors are doing. This section involves a bit of linear algebra, though you can probably complete it even without being expert in linear algebra, and we encourage you to give it a shot. This portion of the notebook is optional and is not graded. 

Lets first see how the GloVe word embeddings relate to gender. You will first compute a vector $g = e_{woman}-e_{man}$, where $e_{woman}$ represents the word vector corresponding to the word *woman*, and $e_{man}$ corresponds to the word vector corresponding to the word *man*. The resulting vector $g$ roughly encodes the concept of "gender". (You might get a more accurate representation if you compute $g_1 = e_{mother}-e_{father}$, $g_2 = e_{girl}-e_{boy}$, etc. and average over them. But just using $e_{woman}-e_{man}$ will give good enough results for now.) 


In [0]:
g = word_to_vec_map['woman'] - word_to_vec_map['man']
print(g)

Now, you will consider the cosine similarity of different words with $g$. Consider what a positive value of similarity means vs a negative cosine similarity. 

In [0]:
print ('List of names and their similarities with constructed vector:')

# girls and boys name
name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for w in name_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

As you can see, female first names tend to have a positive cosine similarity with our constructed vector $g$, while male first names tend to have a negative cosine similarity. This is not suprising, and the result seems acceptable. 

But let's try with some other words.

In [0]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

Do you notice anything surprising? It is astonishing how these results reflect certain unhealthy gender stereotypes. For example, "computer" is closer to "man" while "literature" is closer to "woman". Ouch! 

We'll see below how to reduce the bias of these vectors, using an algorithm due to [Boliukbasi et al., 2016](https://arxiv.org/abs/1607.06520). Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender specific, while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related. You will have to treat these two type of words differently when debiasing.

### 3.1 - Neutralize bias for non-gender specific words 

The figure below should help you visualize what neutralizing does. If you're using a 50-dimensional word embedding, the 50 dimensional space can be split into two parts: The bias-direction $g$, and the remaining 49 dimensions, which we'll call $g_{\perp}$. In linear algebra, we say that the 49 dimensional $g_{\perp}$ is perpendicular (or "othogonal") to $g$, meaning it is at 90 degrees to $g$. The neutralization step takes a vector such as $e_{receptionist}$ and zeros out the component in the direction of $g$, giving us $e_{receptionist}^{debiased}$. 

Even though $g_{\perp}$ is 49 dimensional, given the limitations of what we can draw on a screen, we illustrate it using a 1 dimensional axis below. 

<img src="https://github.com/wronnyhuang/word_embed_demo/blob/master/images/neutral.png?raw=1" style="width:800px;height:300px;">
<caption><center> **Figure 2**: The word vector for "receptionist" represented before and after applying the neutralize operation. </center></caption>

**Exercise**: Implement `neutralize()` to remove the bias of words such as "receptionist" or "scientist". Given an input embedding $e$, you can use the following formulas to compute $e^{debiased}$: 

$$e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g\tag{2}$$
$$e^{debiased} = e - e^{bias\_component}\tag{3}$$

If you are an expert in linear algebra, you may recognize $e^{bias\_component}$ as the projection of $e$ onto the direction $g$. If you're not an expert in linear algebra, don't worry about this.

<!-- 
**Reminder**: a vector $u$ can be split into two parts: its projection over a vector-axis $v_B$ and its projection over the axis orthogonal to $v$:
$$u = u_B + u_{\perp}$$
where : $u_B = $ and $ u_{\perp} = u - u_B $
!--> 

In [0]:
def neutralize(word, g, word_to_vec_map):
    """
    Removes the bias of "word" by projecting it on the space orthogonal to the bias axis. 
    This function ensures that gender neutral words are zero in the gender subspace.
    
    Arguments:
        word -- string indicating the word to debias
        g -- numpy-array of shape (50,), corresponding to the bias axis (such as gender)
        word_to_vec_map -- dictionary mapping words to their corresponding vectors.
    
    Returns:
        e_debiased -- neutralized word vector representation of the input "word"
    """
    
    ### START CODE HERE ###
    # Select word vector representation of "word". Use word_to_vec_map. (≈ 1 line)
    e = None
    
    # Compute e_biascomponent using the formula give above. (≈ 1 line)
    e_biascomponent = None
 
    # Neutralize e by substracting e_biascomponent from it 
    # e_debiased should be equal to its orthogonal projection. (≈ 1 line)
    e_debiased = None
    ### END CODE HERE ###
    
    return e_debiased

In [0]:
e = "receptionist"
print("cosine similarity between " + e + " and g, before neutralizing: ", cosine_similarity(word_to_vec_map["receptionist"], g))

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print("cosine similarity between " + e + " and g, after neutralizing: ", cosine_similarity(e_debiased, g))

**Expected Output**: The second result is essentially 0, up to numerical roundof (on the order of $10^{-17}$).


<table>
    <tr>
        <td>
            **cosine similarity between receptionist and g, before neutralizing:** :
        </td>
        <td>
         0.330779417506
        </td>
    </tr>
        <tr>
        <td>
            **cosine similarity between receptionist and g, after neutralizing:** :
        </td>
        <td>
         -3.26732746085e-17
    </tr>
</table>

### 3.2 - Equalization algorithm for gender-specific words

Next, lets see how debiasing can also be applied to word pairs such as "actress" and "actor." Equalization is applied to pairs of words that you might want to have differ only through the gender property. As a concrete example, suppose that "actress" is closer to "babysit" than "actor." By applying neutralizing to "babysit" we can reduce the gender-stereotype associated with babysitting. But this still does not guarantee that "actor" and "actress" are equidistant from "babysit." The equalization algorithm takes care of this. 

The key idea behind equalization is to make sure that a particular pair of words are equi-distant from the 49-dimensional $g_\perp$. The equalization step also ensures that the two equalized steps are now the same distance from $e_{receptionist}^{debiased}$, or from any other work that has been neutralized. In pictures, this is how equalization works: 

<img src="https://github.com/wronnyhuang/word_embed_demo/blob/master/images/equalize10.png?raw=1" style="width:800px;height:400px;">


The derivation of the linear algebra to do this is a bit more complex. (See Bolukbasi et al., 2016 for details.) But the key equations are: 

$$ \mu = \frac{e_{w1} + e_{w2}}{2}\tag{4}$$ 

$$ \mu_{B} = \frac {\mu \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{5}$$ 

$$\mu_{\perp} = \mu - \mu_{B} \tag{6}$$

$$ e_{w1B} = \frac {e_{w1} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{7}$$ 
$$ e_{w2B} = \frac {e_{w2} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{8}$$


$$e_{w1B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w1B}} - \mu_B} {|(e_{w1} - \mu_{\perp}) - \mu_B)|} \tag{9}$$


$$e_{w2B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w2B}} - \mu_B} {|(e_{w2} - \mu_{\perp}) - \mu_B)|} \tag{10}$$

$$e_1 = e_{w1B}^{corrected} + \mu_{\perp} \tag{11}$$
$$e_2 = e_{w2B}^{corrected} + \mu_{\perp} \tag{12}$$


**Exercise**: Implement the function below. Use the equations above to get the final equalized version of the pair of words. Good luck!

In [0]:
def equalize(pair, bias_axis, word_to_vec_map):
    """
    Debias gender specific words by following the equalize method described in the figure above.
    
    Arguments:
    pair -- pair of strings of gender specific words to debias, e.g. ("actress", "actor") 
    bias_axis -- numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    word_to_vec_map -- dictionary mapping words to their corresponding vectors
    
    Returns
    e_1 -- word vector corresponding to the first word
    e_2 -- word vector corresponding to the second word
    """
    
    ### START CODE HERE ###
    # Step 1: Select word vector representation of "word". Use word_to_vec_map. (≈ 2 lines)
    w1, w2 = None
    e_w1, e_w2 = None
    
    # Step 2: Compute the mean of e_w1 and e_w2 (≈ 1 line)
    mu = None

    # Step 3: Compute the projections of mu over the bias axis and the orthogonal axis (≈ 2 lines)
    mu_B = None
    mu_orth = None

    # Step 4: Use equations (7) and (8) to compute e_w1B and e_w2B (≈2 lines)
    e_w1B = None
    e_w2B = None
        
    # Step 5: Adjust the Bias part of e_w1B and e_w2B using the formulas (9) and (10) given above (≈2 lines)
    corrected_e_w1B = None
    corrected_e_w2B = None

    # Step 6: Debias by equalizing e1 and e2 to the sum of their corrected projections (≈2 lines)
    e1 = None
    e2 = None
                                                                
    ### END CODE HERE ###
    
    return e1, e2

In [0]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], g))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], g))
print()
e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

**Expected Output**:

cosine similarities before equalizing:
<table>
    <tr>
        <td>
            **cosine_similarity(word_to_vec_map["man"], gender)** =
        </td>
        <td>
         -0.117110957653
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(word_to_vec_map["woman"], gender)** =
        </td>
        <td>
         0.356666188463
        </td>
    </tr>
</table>

cosine similarities after equalizing:
<table>
    <tr>
        <td>
            **cosine_similarity(u1, gender)** =
        </td>
        <td>
         -0.700436428931
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(u2, gender)** =
        </td>
        <td>
         0.700436428931
        </td>
    </tr>
</table>

Please feel free to play with the input words in the cell above, to apply equalization to other pairs of words. 

These debiasing algorithms are very helpful for reducing bias, but are not perfect and do not eliminate all traces of bias. For example, one weakness of this implementation was that the bias direction $g$ was defined using only the pair of words _woman_ and _man_. As discussed earlier, if $g$ were defined by computing $g_1 = e_{woman} - e_{man}$; $g_2 = e_{mother} - e_{father}$; $g_3 = e_{girl} - e_{boy}$; and so on and averaging over them, you would obtain a better estimate of the "gender" dimension in the 50 dimensional word embedding space. Feel free to play with such variants as well.  
                     

### Congratulations

You have come to the end of this notebook, and have seen a lot of the ways that word vectors can be used as well as  modified. 

Congratulations on finishing this notebook! 


**References**:
- The debiasing algorithm is from Bolukbasi et al., 2016, [Man is to Computer Programmer as Woman is to
Homemaker? Debiasing Word Embeddings](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf)
- The GloVe word embeddings were due to Jeffrey Pennington, Richard Socher, and Christopher D. Manning. (https://nlp.stanford.edu/projects/glove/)
